In [1]:
# 作业要求：由每天前9个小时的18个空气的影响因素（如：NO，CO，SO2，PM2.5等等）来预测第10个小时的PM2.5，
# train.csv是一年的数据，每个月取了20天，每天24小时
# 环境：Anaconda3 +Jupyter Notebook，python3.6.8

In [2]:
# 0.开始之前先导入需要的库
import sys
# 该模块提供对解释器使用或维护的一些变量的访问，以及与解释器强烈交互的函数
import pandas as pd
# 一个强大的分析结构化数据的工具集
import numpy as np
# Python的一个扩展程序库，支持大量的维度数组与矩阵运算
import math
# 数学运算的库

In [3]:
# 1.导入train数据
data = pd.read_csv('./train.csv', encoding = 'big5')
print(data)
# 数据说明：每行代表一个观测数据在24个时辰的值 故有效数据共24列，加上前三列标注，共27列；
# 每天有18个观测数据类型，取每个月前20天的数据为训练集，每年12个月，共18*20*12=4320行。所以数据为4320*27

              日期  測站          測項     0     1     2     3     4     5     6  \
0       2014/1/1  豐原    AMB_TEMP    14    14    14    13    12    12    12   
1       2014/1/1  豐原         CH4   1.8   1.8   1.8   1.8   1.8   1.8   1.8   
2       2014/1/1  豐原          CO  0.51  0.41  0.39  0.37  0.35   0.3  0.37   
3       2014/1/1  豐原        NMHC   0.2  0.15  0.13  0.12  0.11  0.06   0.1   
4       2014/1/1  豐原          NO   0.9   0.6   0.5   1.7   1.8   1.5   1.9   
...          ...  ..         ...   ...   ...   ...   ...   ...   ...   ...   
4315  2014/12/20  豐原         THC   1.8   1.8   1.8   1.8   1.8   1.7   1.7   
4316  2014/12/20  豐原       WD_HR    46    13    61    44    55    68    66   
4317  2014/12/20  豐原  WIND_DIREC    36    55    72   327    74    52    59   
4318  2014/12/20  豐原  WIND_SPEED   1.9   2.4   1.9   2.8   2.3   1.9   2.1   
4319  2014/12/20  豐原       WS_HR   0.7   0.8   1.8     1   1.9   1.7   2.1   

      ...    14    15    16    17    18    19    20    21    22

In [4]:
# 2.数据处理:取需要的数值部分，将'RAINFALL' 栏位全部补0
data = data.iloc[:, 3:]
# 分割出前3列，从第4列开始将数据存到data
data[data == 'NR'] = 0
# 把降雨的NR字符变成数值0
raw_data = data.to_numpy()
# 把dataframe转换成numpy的数组

In [5]:
# 3.提取特征（1）
# step1: 声明一个18维向量（data）
# strp2: 对于 train data 的第i行 Data[i_th row%18].append(every element in i_th row)
# 数据要成为像  2014/1/1   2014/1/2   2014/1/3 这样的向量
# 将原始4320 * 18 的资料依照每个月重新分组成12 个18 (features) * 480 (hours)（20天每天24小时---训练数据） 的资料。
month_data = {}
for month in range(12):
    sample = np.empty([18, 480])
    for day in range(20):
        sample[:, day * 24 : (day + 1) * 24] = raw_data[18 * (20 * month + day) : 18 * (20 * month + day + 1), :]
    month_data[month] = sample

In [6]:
# 4.提取特征（2）
# 伪代码：
# 声明前9小时数据为train_x，第十个小时的pm2.5声明为train_y
# 对于给定数据中的i：
#     train_x.append(前九小时数据)
#     train_y.append(第十小时pm2.5的值)
# 给train_x中的每个数据添加一个偏差项
x = np.empty([12 * 471, 18 * 9], dtype = float)
y = np.empty([12 * 471, 1], dtype = float)
for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day == 19 and hour > 14:
                continue
            x[month * 471 + day * 24 + hour, :] = month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1, -1) #vector dim:18*9 (9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9)
            y[month * 471 + day * 24 + hour, 0] = month_data[month][9, day * 24 + hour + 9] #value
print(x)
print(y)
# 说明：每个月会有480hrs，每9小时形成一个data，每个月会有471 个data，故总资料数为471 * 12 笔，
# 而每笔data 有9 * 18 的features (一小时18 个features * 9 小时)。
# 对应的target 则有471 * 12 个(第10 个小时的PM2.5)

[[14.  14.  14.  ...  2.   2.   0.5]
 [14.  14.  13.  ...  2.   0.5  0.3]
 [14.  13.  12.  ...  0.5  0.3  0.8]
 ...
 [17.  18.  19.  ...  1.1  1.4  1.3]
 [18.  19.  18.  ...  1.4  1.3  1.6]
 [19.  18.  17.  ...  1.3  1.6  1.8]]
[[30.]
 [41.]
 [44.]
 ...
 [17.]
 [24.]
 [29.]]


In [7]:
# 5.归一化处理
# 从训练集中取出一部分设立验证集，目的是为了对模型进行验证
mean_x = np.mean(x, axis = 0) #18 * 9 
std_x = np.std(x, axis = 0) #18 * 9 
for i in range(len(x)): #12 * 471
    for j in range(len(x[0])): #18 * 9 
        if std_x[j] != 0:
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]

In [8]:
# 6.将训练数据分割为“train_set”和“validation_set”
# 生成比较中用来训练的train_set 和不会被放入训练、只是用来验证的validation_set
x_train_set = x[: math.floor(len(x) * 0.8), :]
y_train_set = y[: math.floor(len(y) * 0.8), :]
x_validation = x[math.floor(len(x) * 0.8): , :]
y_validation = y[math.floor(len(y) * 0.8): , :]
print(x_train_set)
print(y_train_set)
print(x_validation)
print(y_validation)
print(len(x_train_set))
print(len(y_train_set))
print(len(x_validation))
print(len(y_validation))

[[-1.35825331 -1.35883937 -1.359222   ...  0.26650729  0.2656797
  -1.14082131]
 [-1.35825331 -1.35883937 -1.51819928 ...  0.26650729 -1.13963133
  -1.32832904]
 [-1.35825331 -1.51789368 -1.67717656 ... -1.13923451 -1.32700613
  -0.85955971]
 ...
 [ 0.86929969  0.70886668  0.38952809 ...  1.39110073  0.2656797
  -0.39079039]
 [ 0.71018876  0.39075806  0.07157353 ...  0.26650729 -0.39013211
  -0.39079039]
 [ 0.3919669   0.07264944  0.07157353 ... -0.38950555 -0.39013211
  -0.85955971]]
[[30.]
 [41.]
 [44.]
 ...
 [ 7.]
 [ 5.]
 [14.]]
[[ 0.07374504  0.07264944  0.07157353 ... -0.38950555 -0.85856912
  -0.57829812]
 [ 0.07374504  0.07264944  0.23055081 ... -0.85808615 -0.57750692
   0.54674825]
 [ 0.07374504  0.23170375  0.23055081 ... -0.57693779  0.54674191
  -0.1095288 ]
 ...
 [-0.88092053 -0.72262212 -0.56433559 ... -0.57693779 -0.29644471
  -0.39079039]
 [-0.7218096  -0.56356781 -0.72331287 ... -0.29578943 -0.39013211
  -0.1095288 ]
 [-0.56269867 -0.72262212 -0.88229015 ... -0.3895055

In [9]:
# 7.训练
# 设置超参数：学习率，迭代次数等
# 计算损失L
# 计算梯度gradient
# 梯度下降
# Adagrad:损失函数采用均方根误差公式->对参数w计算梯度值，梯度下降，采用RMSprop(指数加权移动平均数)
dim = 18 * 9 + 1
# 因为存在偏差bias，所以dim+1
w = np.zeros([dim, 1])
# w维度为163*1
x = np.concatenate((np.ones([12 * 471, 1]), x), axis = 1).astype(float)
# x_train_set维度为 4521*163
learning_rate = 100
# 设置学习率
iter_time = 1000
# 设置迭代数
adagrad = np.zeros([dim, 1])
#RMSprop参数初始化
eps = 0.0000000001
for t in range(iter_time):  # 迭代
    loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y, 2))/471/12)#rmse
    if(t%100==0):
        print(str(t) + ":" + str(loss))
    gradient = 2 * np.dot(x.transpose(), np.dot(x, w) - y) #dim*1
    # 计算梯度值
    adagrad += gradient ** 2
    w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
    # #更新参数w
np.save('weight.npy', w)
# 保存参数w

0:27.071214829194115
100:33.78905859777454
200:19.91375129819709
300:13.531068193689686
400:10.645466158446165
500:9.27735345547506
600:8.518042045956497
700:8.014061987588416
800:7.636756824775686
900:7.33656374037112


In [10]:
# 8.测试
# 伪代码
# step1：读取测试数据文件
# 每18行：
#   test_x.append([1])
#   test_x.append(9-hr data)
#   test_y=np.dot(weight vector,test_x)
# 说明：载入test data，并且以相似于训练资料预先处理和特征提取的方式处理，使test data 形成240 个维度为18 * 9 + 1 的资料。
testdata = pd.read_csv('./test.csv', header = None, encoding = 'big5')
# 读入测试数据test.csv
test_data = testdata.iloc[:, 2:]
# 丢弃前2列，需要的是从第3列开始的数据
test_data[test_data == 'NR'] = 0
# 把将于的NR字符变成数字8
test_data = test_data.to_numpy()
# 将dataframe变成numpy数组
test_x = np.empty([240, 18*9], dtype = float)
# 将test数据也变成 240 个维度为 18 * 9 + 1 的数据。
for i in range(240):
    test_x[i, :] = test_data[18 * i: 18* (i + 1), :].reshape(1, -1)
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        if std_x[j] != 0:
            test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]
test_x = np.concatenate((np.ones([240, 1]), test_x), axis = 1).astype(float)
print(test_x)

[[ 1.         -0.24447681 -0.24545919 ... -0.67065391 -1.04594393
   0.07797893]
 [ 1.         -1.35825331 -1.51789368 ...  0.17279117 -0.10906991
  -0.48454426]
 [ 1.          1.5057434   1.34508393 ... -1.32666675 -1.04594393
  -0.57829812]
 ...
 [ 1.          0.3919669   0.54981237 ...  0.26650729 -0.20275731
   1.20302531]
 [ 1.         -1.8355861  -1.8360023  ... -1.04551839 -1.13963133
  -1.14082131]
 [ 1.         -1.35825331 -1.35883937 ...  2.98427476  3.26367657
   1.76554849]]


<ipython-input-10-370bc42a7637>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data[test_data == 'NR'] = 0
D:\software\anaconda\lib\site-packages\pandas\core\frame.py:3089: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


In [11]:
# 9.载入模型
# 伪代码：
# 每18行读取test x.csv文件-》
# test_x.append([1])
# test_x.append(9小时数据)测试y = np。
# test_y=np.dot(weight vector,testx)
w = np.load('weight.npy')
ans_y = np.dot(test_x, w)
print(ans_y)

[[ 5.17496040e+00]
 [ 1.83062143e+01]
 [ 2.04912181e+01]
 [ 1.15239429e+01]
 [ 2.66160568e+01]
 [ 2.05313481e+01]
 [ 2.19065510e+01]
 [ 3.17364687e+01]
 [ 1.33916741e+01]
 [ 6.44564665e+01]
 [ 2.02645688e+01]
 [ 1.53585761e+01]
 [ 6.85894728e+01]
 [ 4.84281137e+01]
 [ 1.87023338e+01]
 [ 1.01885957e+01]
 [ 3.07403629e+01]
 [ 7.11322178e+01]
 [-4.13051739e+00]
 [ 1.82356940e+01]
 [ 3.85789223e+01]
 [ 7.13115197e+01]
 [ 7.41034816e+00]
 [ 1.87179553e+01]
 [ 1.49372503e+01]
 [ 3.67197367e+01]
 [ 1.79616970e+01]
 [ 7.57894629e+01]
 [ 1.23093102e+01]
 [ 5.62953517e+01]
 [ 2.51131609e+01]
 [ 4.61024867e+00]
 [ 2.48377055e+00]
 [ 2.47594223e+01]
 [ 3.04802805e+01]
 [ 3.84639307e+01]
 [ 4.42023106e+01]
 [ 3.00868360e+01]
 [ 4.04736750e+01]
 [ 2.92264799e+01]
 [ 5.60645605e+00]
 [ 3.86660161e+01]
 [ 3.46102134e+01]
 [ 4.83896975e+01]
 [ 1.47572477e+01]
 [ 3.44668201e+01]
 [ 2.74831069e+01]
 [ 1.20008794e+01]
 [ 2.13780362e+01]
 [ 2.85444031e+01]
 [ 2.01655138e+01]
 [ 1.07966781e+01]
 [ 2.2171035

In [12]:
# 10.将预测保存到csv文件
import csv
with open('submit.csv', mode='w', newline='') as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['id', 'value']
    print(header)
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id_' + str(i), ans_y[i][0]]
        csv_writer.writerow(row)
        print(row)

['id', 'value']
['id_0', 5.17496039898473]
['id_1', 18.306214253527884]
['id_2', 20.491218094180553]
['id_3', 11.523942869805396]
['id_4', 26.61605675230615]
['id_5', 20.531348081761205]
['id_6', 21.90655101879739]
['id_7', 31.736468747068834]
['id_8', 13.391674055111714]
['id_9', 64.45646650291955]
['id_10', 20.264568836159434]
['id_11', 15.35857607736122]
['id_12', 68.58947276926725]
['id_13', 48.428113747457175]
['id_14', 18.702333824193207]
['id_15', 10.188595737466695]
['id_16', 30.74036285982043]
['id_17', 71.13221776355115]
['id_18', -4.130517391262446]
['id_19', 18.23569401642868]
['id_20', 38.57892227500776]
['id_21', 71.3115197253133]
['id_22', 7.410348162634058]
['id_23', 18.717955330321416]
['id_24', 14.937250260084554]
['id_25', 36.71973669470531]
['id_26', 17.961697005662685]
['id_27', 75.78946287210539]
['id_28', 12.309310248614453]
['id_29', 56.29535173964959]
['id_30', 25.11316086566151]
['id_31', 4.610248674094034]
['id_32', 2.4837705545150186]
['id_33', 24.7594222613